# Sign Language Recognition

<img src=https://i.imgur.com/qpRACer.png />

# 0. Importing Libraries

In [ ]:
import os
import cv2
import csv
import PIL
import pickle
import numpy as np
import pandas as pd
import mediapipe as mp
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

# 1. Functions for Preprocessing Data

In [ ]:
drawingModule = mp.solutions.drawing_utils
handsModule = mp.solutions.hands
mp_model = handsModule.Hands(static_image_mode = True)

In [ ]:
def input_img(filePath, fileName, fileFormat):
    
    load_path = filePath + fileName + '.' + fileFormat
    save_path = filePath + fileName
    
    return load_path, save_path

In [ ]:
def mediapipe_detection(fileName, mp_model):
    
    image = cv2.imread(fileName)
    results = mp_model.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    if results.multi_hand_landmarks != None:
        for handLandmarks in results.multi_hand_landmarks:
            drawingModule.draw_landmarks(image, handLandmarks, handsModule.HAND_CONNECTIONS, 
                                         drawingModule.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4), 
                                         drawingModule.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2)
                                        )

        cv2.imshow(fileName, image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    img = PIL.Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    
    return img, results

In [ ]:
def hand_detection(results):
    
    hand = 'none'

    if len(results.multi_handedness) == 1:
        if results.multi_handedness[0].classification[0].label == 'Left':
            hand = 'left'
        if results.multi_handedness[0].classification[0].label == 'Right':
            hand = 'right'
    else:
        hand = 'both'

    return hand

In [ ]:
def extract_keypoints(hand, results):
    
    kp_list = []
    
    if hand == 'right':
        for i in range(0, 21):
            kp_list.append([0, 0, 0])
        
        for data_point in results.multi_hand_landmarks[0].landmark:
            kp_list.append([data_point.x, data_point.y, data_point.z])
        
    if hand == 'left':
        for data_point in results.multi_hand_landmarks[0].landmark:
            kp_list.append([data_point.x, data_point.y, data_point.z])
        
        for i in range(0, 21):
            kp_list.append([0, 0, 0])
    
    if hand == 'both':
        for data_point in results.multi_hand_landmarks[0].landmark:
            kp_list.append([data_point.x, data_point.y, data_point.z])
        
        for data_point in results.multi_hand_landmarks[1].landmark:
            kp_list.append([data_point.x, data_point.y, data_point.z])
    
    keypoints = np.array(kp_list)
    
    return keypoints

In [ ]:
def create_csv(csv_file):
    
    columns = ['letter']
    
    for val in range(1, 22):
        columns += ['xl{}'.format(val), 'yl{}'.format(val), 'zl{}'.format(val)]
    
    for val in range(1, 22):
        columns += ['xr{}'.format(val), 'yr{}'.format(val), 'zr{}'.format(val)]
    
    with open(csv_file, mode='w', newline='') as f:
        csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerow(columns)

In [ ]:
def write_csv(letter, keypoints, csv_file):
    
    temp = list(keypoints.flatten())
    temp.insert(0, letter)
    
    with open(csv_file, mode='a', newline='') as f:
        csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerow(temp)

# 2. CSV file with Keypoint

In [ ]:
# create_csv('features.csv')

In [ ]:
# load_path, save_path = input_img('', 'right', 'jpg')
# img, results = mediapipe_detection(load_path, mp_model)
# hand = hand_detection(results)
# keypoints = extract_keypoints(hand, results)
# write_csv('A', keypoints, 'features.csv')
# np.save(save_path, keypoints)
# np.load(save_path + '.npy')

In [ ]:
# label = 'A'
# count = 97

# for i in range(1, count):
#     load_path, save_path = input_img('test/' + label + '/', label + str(i), 'jpg')
#     img, results = mediapipe_detection(load_path, mp_model)
#     hand = hand_detection(results)
#     keypoints = extract_keypoints(hand, results)
#     write_csv(label, keypoints, 'features.csv')
#     np.save(save_path, keypoints)

# 3. Training

In [ ]:
df = pd.read_csv('features.csv')

X = df.drop('letter', axis=1)
y = df['letter']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1234)

In [ ]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [ ]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

# 4. Testing & Evaluation of Models

In [ ]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))

In [ ]:
fit_models['lr'].predict(X_test)

In [ ]:
y_test

In [ ]:
with open('best_model.pkl', 'wb') as f:
    pickle.dump(fit_models['lr'], f)

# 5. Camera Demonstration

In [ ]:
with open('best_model.pkl', 'rb') as f:
    best_model = pickle.load(f)

In [ ]:
cam = cv2.VideoCapture(0)

while cam.isOpened():
    
    try:
        ret, frame = cam.read()
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        image.flags.writeable = False
        
        results = mp_model.process(image)
        
        image.flags.writeable = True
        
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.multi_hand_landmarks != None:
            for handLandmarks in results.multi_hand_landmarks:
                drawingModule.draw_landmarks(image, handLandmarks, handsModule.HAND_CONNECTIONS, 
                                             drawingModule.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4), 
                                             drawingModule.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2)
                                            )
        
        hand = 'none'
        
        if len(results.multi_handedness) == 1:
            if results.multi_handedness[0].classification[0].label == 'Left':
                hand = 'left'
            if results.multi_handedness[0].classification[0].label == 'Right':
                hand = 'right'
        else:
            hand = 'both'
        
        
        kp_list = []
        
        if hand == 'right':
            for i in range(0, 21):
                kp_list.append([0, 0, 0])
            
            for data_point in results.multi_hand_landmarks[0].landmark:
                kp_list.append([data_point.x, data_point.y, data_point.z])
        
        if hand == 'left':
            for data_point in results.multi_hand_landmarks[0].landmark:
                kp_list.append([data_point.x, data_point.y, data_point.z])
            
            for i in range(0, 21):
                kp_list.append([0, 0, 0])
        
        if hand == 'both':
            for data_point in results.multi_hand_landmarks[0].landmark:
                kp_list.append([data_point.x, data_point.y, data_point.z])
            
            for data_point in results.multi_hand_landmarks[1].landmark:
                kp_list.append([data_point.x, data_point.y, data_point.z])
        
        keypoints = np.array(kp_list)
        temp = list(keypoints.flatten())
        
        test = pd.DataFrame([temp])
        hand_pred = best_model.predict(test)[0]
        hand_prob = best_model.predict_proba(test)[0]
        
        cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
        cv2.putText(image, 'LETTER', (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, hand_pred.split(' ')[0], (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(image, 'PROB', (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(round(np.amax(hand_prob), 2)), (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
    except:
        pass
    
    cv2.imshow('Camera Demonstration', image)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cam.release()
cv2.destroyAllWindows()

# 6. Image Demonstration

In [ ]:
fileName = 'a.jpg'
image = cv2.imread(fileName)
results = mp_model.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

if results.multi_hand_landmarks != None:
    for handLandmarks in results.multi_hand_landmarks:
        drawingModule.draw_landmarks(image, handLandmarks, handsModule.HAND_CONNECTIONS, 
                                     drawingModule.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4), 
                                     drawingModule.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2)
                                    )

img = PIL.Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

hand = 'none'

if len(results.multi_handedness) == 1:
    if results.multi_handedness[0].classification[0].label == 'Left':
        hand = 'left'
    if results.multi_handedness[0].classification[0].label == 'Right':
        hand = 'right'
else:
    hand = 'both'
        

kp_list = []

if hand == 'right':
    for i in range(0, 21):
        kp_list.append([0, 0, 0])
    
    for data_point in results.multi_hand_landmarks[0].landmark:
        kp_list.append([data_point.x, data_point.y, data_point.z])

if hand == 'left':
    for data_point in results.multi_hand_landmarks[0].landmark:
        kp_list.append([data_point.x, data_point.y, data_point.z])
    
    for i in range(0, 21):
        kp_list.append([0, 0, 0])

if hand == 'both':
    for data_point in results.multi_hand_landmarks[0].landmark:
        kp_list.append([data_point.x, data_point.y, data_point.z])
    
    for data_point in results.multi_hand_landmarks[1].landmark:
        kp_list.append([data_point.x, data_point.y, data_point.z])

keypoints = np.array(kp_list)
temp = list(keypoints.flatten())

test = pd.DataFrame([temp])
hand_pred = best_model.predict(test)[0]
hand_prob = best_model.predict_proba(test)[0]

cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
cv2.putText(image, 'LETTER', (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
cv2.putText(image, hand_pred.split(' ')[0], (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
cv2.putText(image, 'PROB', (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
cv2.putText(image, str(round(np.amax(hand_prob), 2)), (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

cv2.imshow(fileName, image)
cv2.waitKey(0)
cv2.destroyAllWindows()